In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TestApp").master("yarn").config("spark.cores.max", "4").config("spark.sql.shuffle.partitions", "5").getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/30 22:13:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/30 22:13:38 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [27]:
FILE_PATH = "/movies/ratings.csv"

rating_df = spark.read.csv(FILE_PATH, header=True, inferSchema=True)
rating_df.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [28]:
ratings.createOrReplaceTempView("ratings")

result = spark.sql(
    """
    SELECT movieId, ROUND(AVG(rating), 3) AS avgRating, COUNT(*) AS numberOfRatings 
    FROM ratings
    GROUP BY movieId
    ORDER BY numberOfRatings DESC
    """
)

# Wyświetlenie wyników
result.show()

+-------+---------+---------------+
|movieId|avgRating|numberOfRatings|
+-------+---------+---------------+
|    356|    4.164|            329|
|    318|    4.429|            317|
|    296|    4.197|            307|
|    593|    4.161|            279|
|   2571|    4.192|            278|
|    260|    4.231|            251|
|    480|     3.75|            238|
|    110|    4.032|            237|
|    589|    3.971|            224|
|    527|    4.225|            220|
|   2959|    4.273|            218|
|      1|    3.921|            215|
|   1196|    4.216|            211|
|   2858|    4.056|            204|
|     50|    4.238|            204|
|     47|    3.975|            203|
|    780|    3.446|            202|
|    150|    3.846|            201|
|   1198|    4.208|            200|
|   4993|    4.106|            198|
|   1210|    4.138|            196|
|    858|    4.289|            192|
|    457|    3.992|            190|
|    592|    3.429|            189|
|   2028|    4.146|         

In [40]:
FILE_PATH = "/movies/ratings.csv"
rating_df = spark.read.csv(FILE_PATH, header=True, inferSchema=True)

FILE_PATH_MOVIE = "/movies/movies.csv"
movies_df = spark.read.csv(FILE_PATH_MOVIE, header=True, inferSchema=True)



# Wyświetlenie wyników
# rating_df.show(20)

# Wyświetlenie wyników
# movies_df.show(20)


sql_query = """
    SELECT r.movieId AS ID_FILMU, m.title AS TYTUL_FILMU, ROUND(AVG(r.rating), 3) AS SREDNIA_OCENA, COUNT(r.rating) AS numberOfRatings
    FROM ratings r
    JOIN movies m ON r.movieId = m.movieId
    GROUP BY r.movieId, m.title
    ORDER BY numberOfRatings DESC
"""


result = spark.sql(sql_query)


result.show()

+--------+--------------------+-------------+---------------+
|ID_FILMU|         TYTUL_FILMU|SREDNIA_OCENA|numberOfRatings|
+--------+--------------------+-------------+---------------+
|     356| Forrest Gump (1994)|        4.164|            329|
|     318|Shawshank Redempt...|        4.429|            317|
|     296| Pulp Fiction (1994)|        4.197|            307|
|     593|Silence of the La...|        4.161|            279|
|    2571|  Matrix, The (1999)|        4.192|            278|
|     260|Star Wars: Episod...|        4.231|            251|
|     480|Jurassic Park (1993)|         3.75|            238|
|     110|   Braveheart (1995)|        4.032|            237|
|     589|Terminator 2: Jud...|        3.971|            224|
|     527|Schindler's List ...|        4.225|            220|
|    2959|   Fight Club (1999)|        4.273|            218|
|       1|    Toy Story (1995)|        3.921|            215|
|    1196|Star Wars: Episod...|        4.216|            211|
|      5